In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.fftpack as fft
from astropy.io import fits
from scipy import optimize
from poppy import zernike
from scipy import signal
from scipy import interpolate
import os
import pytz
import time
import re

# Telescope Dimensions

In [2]:
outD = 7.77010            # primary diameter (m)
inD = 1.024               # inner M2 diameter (m)
n = 48                    # number sample points across the screen (Not the number of subapertures)
nacross = 43              # number of subapertures across the aperture
pscale = outD/(nacross)   # pixel size (m) of samples in pupil plane

# Define Aperture

In [3]:
#  Aperture containing zeros
x = np.linspace(-(n)/2,(n)/2,n)*pscale 
y = np.linspace(-(n)/2,(n)/2,n)*pscale
mg = np.meshgrid(x,y)
ar = np.sqrt(np.sum((m**2 for m in mg)))
ap_outer = (ar <= (7.3)/2) #slightly oversized because GPI does not correct teh boundaries well
ap_inner = (ar <= 1.5/2)   

#ap_outer = (ar <= outD/2)
#ap_inner = (ar <= inD/2)   
ap = (ap_outer ^ ap_inner).astype(np.float)

#  Aperture containing nans
ap_nan = np.copy(ap.astype(np.float))  
ap_nan[np.where(ap==0)] = np.nan

/home/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


# Make the spatial freq grid

Will use to plot against spatial PSDs

In [4]:
kx = fft.fftshift(fft.fftfreq(n,pscale))
ky = fft.fftshift(fft.fftfreq(n,pscale))
mg = np.meshgrid(kx,ky)
kr = np.sqrt(np.sum((m**2 for m in mg))) 

/home/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


# Phase Sample

1. name_list - list containing telemetry date strings
2. fname_list - list containing path to telemetry files

In [5]:
rootdir = '/home/sda/mtallis/PhaseScripts/aotelem/Reduced/'
save_path = '/home/sda/mtallis/Results/c_Eri/'
dstr = time.strftime('%Y%m%d')

In [94]:
with open('/home/sda/mtallis/samples/c_Eri_samples.txt','r') as f:
    sample = f.read().splitlines() #outputs as a list of strings

In [9]:
fname_list = list()
name_list = list()

for i in sample:
    for root, dirs, files in os.walk(rootdir):
        for name in files:
            (base,ext) = os.path.splitext(name)
            if (ext in ('.fits')) and (i in base):
                full_name = os.path.join(root,name)
                print(full_name)
                fname_list.append(full_name)  
                name_list.append(base[11:-12])

/home/sda/mtallis/PhaseScripts/aotelem/Reduced/20141217/aored_When_2014.12.17_23.47.47_poldm_phase.fits
/home/sda/mtallis/PhaseScripts/aotelem/Reduced/20141218/aored_When_2014.12.18_0.6.47_poldm_phase.fits
/home/sda/mtallis/PhaseScripts/aotelem/Reduced/20150128/aored_When_2015.1.28_22.8.6_poldm_phase.fits
/home/sda/mtallis/PhaseScripts/aotelem/Reduced/20150128/aored_When_2015.1.28_22.18.47_poldm_phase.fits
/home/sda/mtallis/PhaseScripts/aotelem/Reduced/20150128/aored_When_2015.1.28_22.42.46_poldm_phase.fits
/home/sda/mtallis/PhaseScripts/aotelem/Reduced/20150128/aored_When_2015.1.28_22.52.1_poldm_phase.fits
/home/sda/mtallis/PhaseScripts/aotelem/Reduced/20150128/aored_When_2015.1.28_23.10.27_poldm_phase.fits
/home/sda/mtallis/PhaseScripts/aotelem/Reduced/20150129/aored_When_2015.1.29_21.37.53_poldm_phase.fits
/home/sda/mtallis/PhaseScripts/aotelem/Reduced/20150129/aored_When_2015.1.29_21.54.49_poldm_phase.fits
/home/sda/mtallis/PhaseScripts/aotelem/Reduced/20150129/aored_When_2015.1.29

# Analysis Functions

1. processs_phase - Imports .fits & removes static aberrations in each phase  
2. radialProfile - Computes mean inside each radial bin starting from center
3. sp_power_spec - Computes time average of 2d DFT^2  
4. temp_power_spec - Computes actuator timeseries DFT^2 and averages over aperture
5. linear fit - fits a power law to psd. Behaves like a line in loglog space

In [10]:
def process_phase(filepath):

    hdulist = fits.open(filepath,memmap=True)
    phase = hdulist[0].data.astype('float')
    avg_phase = np.nanmean(phase*ap_nan,axis=0)  # used to find average zernikes 

    # remove zernikes form cube
    z_basis = zernike.zernike_basis_faster(nterms= 6, npix = 48)
    z_coeff = zernike.opd_expand_nonorthonormal(avg_phase, aperture=ap, nterms=6)
    thin_lens = np.sum(z_coeff[:,None,None]*z_basis[:,:,:],axis=0)

    c_phase = (phase - thin_lens[None,:,:])*ap_nan
    c_phase[np.isnan(c_phase)]=0.
    
    return c_phase

In [11]:
def radialProfile(image, center=None):
    """
    Calculate the avearge radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    ## Calculate the indices from the image
    y,x = np.indices((image.shape)) # first determine radii of all pixels
    
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
     
    r = np.hypot(x - center[0], y - center[1]).astype(np.int) 
    
    n = np.bincount(r.ravel())
    sy = np.bincount(r.ravel(), image.ravel())
    mean = sy/n
    
    return mean

In [13]:
def sp_power_spec(phase):    
    
    timesteps, phx, phy = phase.shape 
    phFT = np.zeros((timesteps,phx,phy), dtype=complex)
    for t in np.arange(timesteps):
        phFT[t,:,:] = fft.fftshift(fft.fft2(phase[t,:,:]))*2.0/ap.sum()
    print('Done with FT')
    
    # compute 2d psd cube
    psd2D = np.zeros((timesteps, phx, phy),dtype=float)
    for k in np.arange(phx):
        for l in np.arange(phy):
            psd2D[:,k,l] = np.square(np.abs(phFT[:,k,l]))
    
    avg_psd2D = np.mean(psd2D, axis=0)
    print('Done with PSD')    
    
    # compute radial average of 2d psd cube and frequency
    avg_psd1D =  radialProfile(avg_psd2D)
    
    return avg_psd1D

In [14]:
def temp_power_spec(Y,sample_spacing):
    
    n = len(Y)
    dT = sample_spacing

    w = signal.blackman(n)
    P = np.fft.rfft(Y*w)
    norm = 2.0/n
    P = P * norm

    P2 = np.square(np.abs(P))
    k = np.fft.rfftfreq(n,dT)
    
    return k,P2

In [15]:
def linear_fit(k,Y,sampling_frequency = .001):
    
    if sampling_frequency == .001:
        
        n = len(Y)
        dT = sampling_frequency
        par = np.polyfit(np.log10(k[(k>1.0)]), np.log10(k[(k>1.0)]), 1)
        slope = par[0]
        intercept = par[1]
        print(slope,intercept)
    
    else:
  
        par = np.polyfit(np.log10(k[(k>1./(3.0)) & (k<1./(0.5))]), np.log10(Y[(k>1./(3.0)) & (k<1./(0.5))]), 1)
        slope = par[0]
        intercept = par[1]
        print(slope,intercept)

    return slope,intercept

# Perform Analysis 

Define spatial frequencies (34 bins)

In [ ]:
k = radialProfile(kr)

Store spatial PSD values in DataFrame for later analysis

In [ ]:
df_sp_psd = pd.DataFrame(columns = name_list)

i=0
for file in fname_list:
    print(file)
    print(name_list[i])
    y_2D = process_phase(file)
    y_psd = sp_power_spec(y_2D)
    df_sp_psd[name_list[i]] = y_psd
    i=i+1

In [ ]:
df_sp_psd.to_csv(save_path+'sp_psd'+'_'+dstr+'.txt')

Store temporal PSD values in DataFrame for later analysis.
Key = 'telemetry date'
columns = 'kt' & 't_psd' (Note varying timeseries lengths) 

In [95]:
iterables = [name_list,['kt','t_psd']]
index = pd.MultiIndex.from_product(iterables)
df_t_psd = pd.DataFrame(columns = index)

i=0
for file in fname_list:
    y_2D = process_phase(file)
    y_1D = np.mean(y_2D,axis=(1,2))
    kt,y_psd = temp_power_spec(y_1D,.001)
    y_smoothed = 10**signal.savgol_filter(np.log10(y_psd), 101, 5) 
    df_t_psd[name_list[i],'kt'] = pd.Series(kt)
    df_t_psd[name_list[i],'t_psd'] = pd.Series(y_smoothed)
    print(name_list[i])
    i=i+1

/home/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: Mean of empty slice
/home/anaconda3/lib/python3.5/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


2014.12.17_23.47.47
2014.12.18_0.6.47
2015.1.28_22.8.6
2015.1.28_22.18.47
2015.1.28_22.42.46
2015.1.28_22.52.1
2015.1.28_23.10.27
2015.1.29_21.37.53
2015.1.29_21.54.49
2015.1.29_22.30.16
2015.1.30_21.41.53
2015.1.30_22.22.15
2015.1.30_22.36.5
2015.1.30_23.9.21
2015.8.30_6.31.37
2015.8.30_6.47.34
2015.8.30_6.58.42
2015.8.30_7.13.12
2015.8.30_7.33.49
2015.8.31_6.47.24
2015.8.31_6.58.19
2015.8.31_7.10.3
2015.8.31_7.22.9
2015.8.31_7.39.0
2015.9.1_5.41.31
2015.9.1_5.44.29
2015.9.1_5.56.16
2015.9.1_6.2.31
2015.9.1_6.10.44
2015.9.1_6.16.53
2015.9.1_6.28.6
2015.9.1_6.36.5
2015.9.1_6.53.10
2015.9.1_7.4.0
2015.9.1_7.13.49
2015.9.1_7.15.2
2015.9.1_7.25.12
2015.9.1_7.35.17
2015.11.5_5.37.38
2015.11.5_5.43.35
2015.11.5_5.50.51
2015.11.5_5.55.26
2015.11.5_5.58.56
2015.11.5_6.3.7
2015.11.6_1.51.52
2015.11.6_1.59.20
2015.11.6_2.6.21
2015.11.6_2.13.25
2015.11.6_2.19.5
2015.11.6_2.21.11
2015.11.6_2.29.32
2015.11.6_2.35.25
2015.11.6_2.38.12
2015.11.6_2.41.33
2015.11.6_2.45.24
2015.11.6_2.48.59
2015.11.6_

In [97]:
df_t_psd.to_csv(save_path+'t_psd'+'_'+dstr+'.txt')

# Grouping of samples 

In [ ]:
df = pd.DataFrame({'fname_list':fname_list,'name_list':name_list})

In [ ]:
df['dts'] = pd.to_datetime(df['name_list'],format='%Y.%m.%d_%H.%M.%S',utc=True)

In [ ]:
g_df = df.groupby(pd.TimeGrouper(key='dts',freq='600T'))

In [ ]:
g_df = df.groupby(pd.TimeGrouper(key='dts',freq='600T'))

In [ ]:
for name,group in g_df:
    if len(group['name_list'].values)>0:
        print(name)
        print(group['name_list'].values)
    else:    
        continue

Plotting of PSDs

In [ ]:
m, intercept = linear_fit(kt,y_smoothed,time = True, space = False)
    plt.loglog(kt,y_smoothed, label = name_list[i])
    plt.loglog(kt[kt>1.0], (10**intercept)*kt[kt>1.]**-1*m)

plt.grid(axis='both')
plt.axvline(x=1)
plt.legend()